# Create an API and a Client -- Server side
## Create an API and a Client

Create an API with 5 endpoints:
* */help* returns with the help/usage of the other 4 endpoints
* 1 endpoint, where one needs to use arguments (key/value pairs)
* 1 endpoint, which returns some data in json format. Either the json should contain a description of what the data is or it should be defined in the /help
* 1 endpoint, which adds extra data to the existing dataset
* 1 endpoint, which replaces data in the existing dataset

The Client should request data from the server and create a table or plot from it.

You can use any dataset for this task. 
E.g. `/home/course/Datasets/owld-datasets/datasets`

You have to prepare for errors or for non-valid queries and need to notify the client about the source of the error!

## How to run this service

* Launch a terminal and enter into the directory, that contains this notebook
* then type
```
preview-nb-api.sh API-worksheet.ipynb 
```
* the first line of output will tell you the URL of the API

In [1]:
import json
import base64
import pandas as pd

Using the provided dataset in this directory

In [126]:
fname = 'GlobalCO2emissions.csv'
df = pd.read_csv(fname, parse_dates=[1])

# Prividing shorter column names
co2 = 'CO2 emissions (CDIAC and UN Population)'
mtonpc = 'Per capita CO2 emissions (CDIAC and UN Population)'

## API

We need to define a global `REQUEST` JSON string that will be replaced on each invocation of the API. We only care about path parameters and query string arguments, so we default those to blank here.

In [20]:
REQUEST = json.dumps({
    'path' : {},
    'args' : {}
})

## Endpoints

I've reworked the notebook of the example server. Changed the data handling, arguments and more in the original API. There were several problems with the provided code, which had to be corrected, to at least function in any way.

### 1. /help

Fullfills the "*/help* returns with the help/usage of the other 4 endpoints" critera

In [111]:
# GET /help
req = json.loads(REQUEST)
args = req['args']

response = {'Message': ('This is the /help endpoint of my API assignment.\n\n' +
                        'There are 4 endpoints besides /help, these are the following:\n' +
                        '1. /api/info/: No arguments needed, prints everything, which should be' +
                        'known about the dataset. This is the original description from the example' +
                        'notebook.\n' +
                        '2. /api/data GET: With a "from" and "until" arguments given, the API prints the' +
                        'entries of the dataset between these two given dates. Should be given in YEARS.' +
                        '3. /api/data POST: Given a "date", a "co2" and "mtonpc" value in the field arguments,' +
                        'the API inserts the given data into the dataset.'
                        '4. /api/data PUT: Given a "date", a "co2" and "mtonpc" value in the field arguments,' +
                        'the API updates the row corresponding to the given date with the given values.'
                       )
           } 
print(json.dumps(response))

{"Message": "This is the /help endpoint of my API assignment.\n\nThere are 4 endpoints besides /help, these are the following:\n1. /api/info/: No arguments needed, prints everything, which should beknown about the dataset. This is the original description from the examplenotebook.\n2. /api/data GET: With a \"from\" and \"until\" arguments given, the API prints theentries of the dataset between these two given dates. Should be given in YEARS.3. /api/data POST: Given a \"date\", a \"co2\" and \"mtonpc\" value in the field arguments,the API inserts the given data into the dataset.4. /api/data PUT: Given a \"date\", a \"co2\" and \"mtonpc\" value in the field arguments,the API updates the row corresponding to the given date with the given values."}


In [27]:
# ResponseInfo GET /help
print(json.dumps({
    "headers" : {
        "Content-Type" : "text"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "text"}, "status": 200}


### 2. /api/info

Fullfills the "1 endpoint, which returns some data in json format. Either the json should contain a description of what the data is or it should be defined in the /help" critera.

Using the description and template inside the example server notebook.

In [31]:
# GET /api/info
req = json.loads(REQUEST)
args = req['args']

response = dict(
    name="Global CO2 emissions - CDIAC and UN Population",
    title="Global CO2 emissions - CDIAC and UN Population",
    dataset_id= 652,
    description=("Per capita CO2 emissions have been calculated based on " +
                 "the combination of global emissions data from CDIAC (described below)," +
                 "and UN Population Prospects data.\r\n\r\nPopulation data was derived from:" +
                 "The History Database of the Global Environment (HYDE) collected the data " +
                 "by earlier publications. For the 'OurWorldInData'-series we used various " +
                 "sources: The data for the period before 1900 are taken from the " +
                 "History Database of the Global Environment (HYDE). The data for the " +
                 "World Population between 1900 and 1940 is taken from the UN puplication " +
                 "'The World at Six Billion'. The annual data for the World Population between " +
                 "1950 and 2015 is taken from the UN's World Population Prospects: The 2015 Revision. " +
                 "It is the series 'Total Population - Both Sexes' online available at: " +
                 "https://esa.un.org/unpd/wpp/.\r\n\r\nEmissions data have been sourced from " +
                 "the Carbon Dioxide Information Analysis Centre (CDIAC) database. Emissions data " +
                 "have been converted from units of carbon to carbon dioxide (CO2) using a conversion " +
                 "factor of 3.67.\r\n\r\nCDIAC denote a \"statistical difference\" component which has " +
                 "been included in this data. This statistical difference represents the difference " +
                 "between estimated global CO2 emissions and the sum of national totals. Estimates of " +
                 "CO2 emissions show that the global total of emissions is not equal to the sum of " +
                 "emissions from all countries. This is introduced in several cases: emissions within " +
                 "international territories, which are included in global totals but not attributed to " +
                 "individual countries; inconsistent national reporting where global import and export " +
                 "data is imbalanced; and differing treatment of non-fuel uses of hydrocarbons.\r\n\r\n" +
                 "Full methodology on global, regional, national and statistical difference estimations can " +
                 "be found in Le Quere et al. (2016): Le Quéré, C., Andrew, R. M., Canadell, J. G., Sitch, S., " +
                 "Korsbakken, J. I., Peters, G. P., ... & Keeling, R. F. (2016). Global carbon budget 2016. " +
                 "Earth System Science Data, 8(2), 605. Available at: doi:10.5194/essd-8-605-2016."),
    sources= {
      "name": "Global CO2 emissions - CDIAC and UN Population",
      "dataPublishedBy": "Carbon Dioxide Information Analysis Center (CDIAC)",
      "dataPublisherSource": ("Boden, T. A. and Andres, R. J.: "+
                              "Global, Regional, and National Fossil-Fuel CO2 Emissions. " +
                              "Oak Ridge National Laboratory, US Department of Energy, Oak Ridge, Tenn., USA (2016)"),
      "link": "http://cdiac.ornl.gov/CO2_Emission/",
      "retrievedDate": "31/07/2017"
    }
)

print(json.dumps(response))

{"name": "Global CO2 emissions - CDIAC and UN Population", "title": "Global CO2 emissions - CDIAC and UN Population", "dataset_id": 652, "description": "Per capita CO2 emissions have been calculated based on the combination of global emissions data from CDIAC (described below),and UN Population Prospects data.\r\n\r\nPopulation data was derived from:The History Database of the Global Environment (HYDE) collected the data by earlier publications. For the 'OurWorldInData'-series we used various sources: The data for the period before 1900 are taken from the History Database of the Global Environment (HYDE). The data for the World Population between 1900 and 1940 is taken from the UN puplication 'The World at Six Billion'. The annual data for the World Population between 1950 and 2015 is taken from the UN's World Population Prospects: The 2015 Revision. It is the series 'Total Population - Both Sexes' online available at: https://esa.un.org/unpd/wpp/.\r\n\r\nEmissions data have been sourc

In [19]:
# ResponseInfo GET /api/info
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### 3. /api/data GET

Fullfills the "1 endpoint, where one needs to use arguments (key/value pairs)" critera

In [106]:
# GET /api/data
req = json.loads(REQUEST)
args = req['args']

if not "from" in args or not "until" in args:
    print ("Missing argument. Expecting both 'from' and 'until'")
else:
    date_from = pd.to_datetime(args["from"][0], format="%Y")
    date_until = pd.to_datetime(args["until"][0], format="%Y")

    if date_from >= date_until:
        response = {"queryParams" : query_params_dict, "message" : "'from' should be an earlier year than 'until'"}
        print(json.dumps(response))
    elif (date_from > df.Year.max()) or (date_until<df.Year.min()):
        query_params_dict = {"from" : args["from"][0], "until" : args["until"][0]}
        response = {
            "queryParams" : query_params_dict,
            "message" : "valid range for 'from' and 'until' is between {} and {} ".format(df['Year'].dt.strftime('%Y').min(), 
                                                                                          df['Year'].dt.strftime('%Y').max())
        }
        print(json.dumps(response))
    else:
        rows = df[(df["Year"] >= date_from) &
                             (df["Year"] <= date_until)][["Year", co2, mtonpc]].copy()
        data_dict = [{"date" : date.strftime('%Y'),
                      "co2 emission" : co2s,
                      "metric tons per capita" : mtonpcs} for date, co2s, mtonpcs in rows.values ]
        query_params_dict = {"from" : args["from"][0], "until" : args["until"][0]}
        response = {
            "queryParams" : query_params_dict, 
            "data" : data_dict
        }
        print(json.dumps(response))

Missing argument. Expecting both 'from' and 'until'


In [32]:
# ResponseInfo GET /api/data
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### 4. /app/data POST

Fullfills the "1 endpoint, which adds extra data to the existing dataset" critera

In [113]:
# POST /api/data
req = json.loads(REQUEST)
args = req['args']

if not "date" in args or not "co2" in args or not "mtonpc" in args:
    response = {
    "Error" : "Missing argument. Expecting 'date', 'co2' and 'mtonpc'"
    }
    print(json.dumps(response))
else:
    date = args["date"][0]
    co2s = float(args["co2"][0])
    mtonpcs = float(args["mtonpc"][0])
    if(df.Year.dt.strftime("%Y")==date).sum():
        response = {
            "Error" : "'date' should not be an existing value"
            }
        print(json.dumps(response))
    else:
        new_row = {'Entity': 'World', 'Year': pd.to_datetime(date), co2: co2s, mtonpc: mtonpcs}
        df = df.append(new_row, ignore_index=True)
        df.to_csv(fname, index=False, index_label=None)
        response = {
            "Message" : "upload successful"
        }
        print(json.dumps(response))

{"Error": "Missing argument. Expecting 'date', 'co2' and 'mtonpc'"}


In [32]:
# ResponseInfo POST /api/data
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    },
    "status" : 200
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### 5. PUT /app/data

Fullfills the "1 endpoint, which replaces data in the existing dataset" critera

In [112]:
# PUT /api/data
req = json.loads(REQUEST)
args = req['args']

if not "date" in args or not "co2" in args or not "mtonpc" in args:
    response = {
    "Error" : "Missing argument. Expecting 'date', 'co2' and 'mtonpc'"
    }
    print(json.dumps(response))
else:
    date = args["date"][0]
    co2s = float(args["co2"][0])
    mtonpcs = float(args["mtonpc"][0])
    if not (df.Year.dt.strftime("%Y")==date).sum():
        response = {
            "Error" : "'date' should be an existing one"
            }
        print(json.dumps(response))
    else:
        df.loc[df.Year.dt.strftime("%Y")==date, co2] = co2s
        df.loc[df.Year.dt.strftime("%Y")==date, mtonpc] = mtonpcs
        df.to_csv(fname, index=False, index_label=None)
        response = {
            "Message" : "upload successful"
        }
        print(json.dumps(response))

{"Error": "Missing argument. Expecting 'date', 'co2' and 'mtonpc'"}


### +1. Using the example code

In [28]:
# GET /api/undef
req = json.loads(REQUEST)
args = req['args']

response = {
    "Error" : "This function is not implemented yet" 
}
print(json.dumps(response))

{"Error": "This function is not implemented yet"}


In [29]:
# GET /:nothingfound

print("You seem lost, try /help")

You seem lost, try /help


In [30]:
# GET /

print("You seem lost, try /help")

You seem lost, try /help
